In [3]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json


Define URLs:

In [4]:
ls_2014 = "http://myneta.info/ls2014/index.php?action=show_winners&sort=default"
ls_2009 = "http://myneta.info/ls2009/index.php?action=show_winners&sort=default"
ls_2004 = "http://myneta.info/loksabha2004/index.php?action=show_winners&sort=default"

Define function to download and create dataframe

In [5]:
def process_winner_link(link, year):
    
    html = requests.get(link)
    doc = pq(html.content)
    trs = doc('table').eq(2).children('tr')

    datalist=[]

    for tr in trs:
        winner_dict = {'year':year}
        for i,td in enumerate(tr.findall('td')):
            if i == 0:
                winner_dict['idnum'] = td.text
            if i == 1:
                winner_dict['name'] = td.findall('a')[1].text
            if i == 2:
                winner_dict['constituency'] = td.text
            if i == 3:
                winner_dict['party'] = td.text
            if i == 4:
                winner_dict['crim_cases'] = td.text
                if td.text == None:
                    winner_dict['crim_cases'] = td.find('span').text
            if i == 5:
                winner_dict['education'] = td.text
            if i == 6:
                winner_dict['assets'] =  td.text.replace(',','').replace('Rs','').lstrip()
            if i == 7:
                winner_dict['liabilities'] =  td.text.replace(',','').replace('Rs','').lstrip()

        datalist.append(winner_dict)

    return pd.DataFrame(datalist)
    

In [143]:
df_winner_14 = process_winner_link(ls_2014, 2014)
df_winner_09 = process_winner_link(ls_2009, 2009)
df_winner_04 = process_winner_link(ls_2004, 2004)
df_winner_master = pd.concat([df_14,df_09,df_04])
df_winner_master.to_csv('loksabha_win.csv')

In [7]:
base_candidate = "http://myneta.info/ls"
suffix_candidate = "/index.php?action=show_candidates&constituency_id="

In [116]:
def get_constituents(year, thelink):
    html = requests.get(thelink)
    doc = pq(html.content)
    trs = doc('table').eq(2).find('tr')
    datalist=[]
    print len(trs)
    for tr in trs:
        new_state = pq(tr)('th').text()
        if new_state != "":
            curr_state = new_state 
            #print "state: %s" % curr_state
        else:
            districts = pq(tr)('td')
            if districts != "":
                for x in districts:
                    dist_name = pq(x)('a').text()
                    if dist_name != "":
                        district_dict = {'year':year, 'state':curr_state}
                        district_dict['district'] = dist_name
                        district_dict['thelink'] = pq(x)('a').attr.href
                        datalist.append(district_dict)

    return pd.DataFrame(datalist)


In [121]:
thelink = 'http://myneta.info/ls2014/'
mynetasite = 'http://myneta.info/'
districts = get_constituents(2014, thelink)

226


In [138]:
#for link in districts.thelink :
    
masterlink = 'http://myneta.info/ls2014/index.php?action=show_candidates&constituency_id=23'
thepage = requests.get(masterlink)
doc = pq(thepage.content)

trs = doc('table').eq(2).children('tr')

datalist = []
for tr in trs:
    winner_dict = {'year':year}
    for i,td in enumerate(tr.findall('td')):
        if i == 0:
            winner_dict['idnum'] = td.text
        if i == 1:
            winner_dict['name'] = td.findall('a')[1].text
        if i == 2:
            winner_dict['constituency'] = td.text
        if i == 3:
            winner_dict['party'] = td.text
        if i == 4:
            winner_dict['crim_cases'] = td.text
            if td.text == None:
                winner_dict['crim_cases'] = td.find('span').text
        if i == 5:
            winner_dict['education'] = td.text
        if i == 6:
            winner_dict['assets'] =  td.text.replace(',','').replace('Rs','').lstrip()
        if i == 7:
            winner_dict['liabilities'] =  td.text.replace(',','').replace('Rs','').lstrip()

    datalist.append(winner_dict)
        
#print trs
#print len(doc('.divTableWithFloatingHeader'))

IndexError: list index out of range